<a href="https://colab.research.google.com/github/donesalmeida/AnalyticsEstudos/blob/main/ProjetoCEPdeRisco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [210]:
import os
import pandas as pd
import re

WORK_DIR = '/content/drive/MyDrive/Colab Notebooks/ProjetoBeneton'

WORK_DIR_INPUT = os.path.join(WORK_DIR, 'input')
WORK_DIR_OUTPUT = os.path.join(WORK_DIR, 'output')

arquivo_planilha = os.path.join(WORK_DIR_INPUT, f'Analise area de risco.xlsx')

arquivo_dne =  os.path.join(WORK_DIR_INPUT, f'Cópia de enderecos.csv')

valor_unitario_visita = 15

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [211]:
#Funções Auxiliares
def tratar_cep(item):
  auxiliar = item["cep"]

  num = re.findall(r'\d+', auxiliar)

  retorno = num[0]

  if len(num)>1:
    retorno = retorno + num[1]

  return retorno


In [212]:
df_arquivo_dne = pd.read_csv(arquivo_dne, sep=';')
df_arquivo_dne["cep"] = df_arquivo_dne["cep"].astype(str)
df_arquivo_dne["cep"] = df_arquivo_dne["cep"].str.strip()
df_arquivo_dne['cep_tratado'] = df_arquivo_dne.apply(tratar_cep, axis=1)


<ipython-input-212-508b30dc9268>:1: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_arquivo_dne = pd.read_csv(arquivo_dne, sep=';')


In [213]:

# Perguntas a serem respondidas
# Quantas vezes os CEPs que sao sinistrados visitaram a tabela de 3000
# Considerando a faixa de CEP 5, quantas vezes eles aparece
# Quais as areas que deveriam ter CEP de risco
# Considerando área de risco, quais os CEPs devemos considrear
# Valores de incentivo por regiao

# Dos sinistros, quantas vezes eles apareceram CEPS exatos e CEP5

# Subir tabela powerbi .... CEPS e repetições

# Separar por furto , roubo, MILHA


In [214]:
# Sinistros
df_raw_sinistros = pd.read_excel(arquivo_planilha, sheet_name = 0)
df_raw_sinistros.rename(columns={"CEP Local do Evento": "cep"}, inplace=True)
df_raw_sinistros["cep"] = df_raw_sinistros["cep"].astype(str)
df_raw_sinistros["cep"] = df_raw_sinistros["cep"].str.strip()
df_raw_sinistros["cep_tratado"] = df_raw_sinistros.apply(tratar_cep, axis=1)

df_raw_sinistros["cep5"] = df_raw_sinistros["cep_tratado"].str.slice(0,5)
df_raw_sinistros.drop(["Tipo de Comunicado", "Transportadora", "CPF do motorista",
                       "Nome do motorista",
                       "cep",
                       "id_incidente",
                       "Ano"], axis=1, inplace=True)

df_raw_sinistros = pd.merge(df_raw_sinistros, df_arquivo_dne, how='left', on='cep_tratado', indicator=True)
df_raw_sinistros.drop(["ibge_cod_uf",
               "ibge_cod_cidade",
               "area_cidade_km2",
               "ddd",
               "Região",
               "cep",
               "logradouro",
               "_merge"], axis=1, inplace=True)

# #Base Completa
df_raw_base_ceps = pd.read_excel(arquivo_planilha, sheet_name = 2)
df_raw_base_ceps.rename(columns={"*End CEP": "cep"}, inplace=True)
df_raw_base_ceps["cep"] = df_raw_base_ceps["cep"].astype(str)
df_raw_base_ceps["cep"] = df_raw_base_ceps["cep"].str.strip()
df_raw_base_ceps["cep_tratado"] = df_raw_base_ceps.apply(tratar_cep, axis=1)
df_raw_base_ceps["cep5"] = df_raw_base_ceps["cep_tratado"].str.slice(0,5)


df_raw_base_ceps = pd.merge(df_raw_base_ceps, df_arquivo_dne, how='left', on='cep_tratado', indicator=True)

df_raw_base_ceps.rename(columns={"uf_x": "uf",
                         "cidade_x": "cidade",
                         "bairro_y": "bairro",
                         "latitude_x": "latitude",
                         "longitude_x": "longitude"}, inplace=True)

df_raw_base_ceps.drop(["_merge",
                       "Região",
                       "ddd",
                       "uf",
                       "bairro_x",
                       "ibge_cod_cidade",
                       "area_cidade_km2",
                       "latitude_y",
                       "logradouro_y",
                       "longitude_y",
                       "ibge_cod_uf"], axis=1, inplace=True)

df_raw_base_ceps_5 = (df_raw_base_ceps
                      .groupby(by=["cep5", "uf_y", "cidade", "bairro"])
                      .agg(qtde_ceps=("cep_tratado", "count"))
                      .reset_index()
                      )
df_raw_base_ceps_5.drop_duplicates(subset="cep5", inplace=True)

In [215]:
def contar_ocorrencias_cep_tratado(item):
  return df_raw_sinistros[df_raw_sinistros["cep_tratado"]==item["cep_tratado"]].shape[0]

df_raw_base_ceps["qtde_ocorrencia"] = df_raw_base_ceps.apply(contar_ocorrencias_cep_tratado, axis=1)

df_raw_base_ceps.sort_values(by="qtde_ocorrencia", ascending=False, inplace=True)

df_raw_base_ceps["valor_unitario_visita"] = valor_unitario_visita

df_raw_base_ceps["total_visita"] = valor_unitario_visita * df_raw_base_ceps["qtde_ocorrencia"]

df_raw_base_ceps.drop(["cep5"], axis=1, inplace=True)

df_raw_base_ceps[df_raw_base_ceps["qtde_ocorrencia"]>0].to_excel(os.path.join(WORK_DIR_OUTPUT, f'sinistro_um_para_um.xlsx'), index=False)

In [216]:
def contar_ocorrencias_cep_5(item):
  return df_raw_sinistros[df_raw_sinistros["cep5"]==item["cep5"]].shape[0]

df_raw_base_ceps_5["qtde_ocorrencia"] = df_raw_base_ceps_5.apply(contar_ocorrencias_cep_5, axis=1)

df_raw_base_ceps_5.sort_values(by="qtde_ocorrencia", ascending=False, inplace=True)

df_raw_base_ceps_5["valor_unitario_visita"] = valor_unitario_visita

df_raw_base_ceps_5["total_visita"] = valor_unitario_visita * df_raw_base_ceps_5["qtde_ocorrencia"]

df_raw_base_ceps_5[df_raw_base_ceps_5["qtde_ocorrencia"]>0].to_excel(os.path.join(WORK_DIR_OUTPUT, f'sinistro_um_para_cinco.xlsx'), index=False)

In [217]:
df_raw_base_ceps[df_raw_base_ceps["qtde_ocorrencia"]==0].to_excel(os.path.join(WORK_DIR_OUTPUT, f'sinistro_um_para_um_sem_ocorrencia.xlsx'), index=False)